In [1]:
import os
import glob
from tqdm import tqdm

import pandas as pd

In [2]:
root = "/media/yesindeed/DATADRIVE1/mount/remote_cse/datasets/multitask-moe/vinbigdata-chest-xray-abnormalities-detection/"

df = pd.read_csv(f"{root}/train.csv")

df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
67909,936fd5cff1c058d39817a08f58b72cae,No finding,14,R1,NaN,NaN,NaN,NaN
67910,ca7e72954550eeb610fe22bf0244b7fa,No finding,14,R1,NaN,NaN,NaN,NaN
67911,aa17d5312a0fb4a2939436abca7f9579,No finding,14,R8,NaN,NaN,NaN,NaN
67912,4b56bc6d22b192f075f13231419dfcc8,Cardiomegaly,3,R8,771.0,979.0,1680.0,1311.0


In [4]:
len(df["image_id"].unique())

15000

In [11]:
dict_out = {"id": [], "exam_id": [], "path": [], "class": [], "exam_class": []}

for i, id in enumerate(df["image_id"].unique()):
    sub_df = df.loc[df["image_id"] == id]

    classes = sub_df["class_name"].unique()

    if "No finding" in classes:
        assert len(classes) == 1
    # assert len(classes) == len(
    #     sub_df), f"For image {id}, got classes {list(classes)}"

    img_path = f"./train-png/{id}.png"

    assert os.path.exists(f"{root}/{img_path.replace('./', '')}")

    cls = ",".join(x.lower() for x in classes)
    cls = cls.replace("no finding", "no findings")

    dict_out["id"].append(id)
    dict_out["exam_id"].append(id)

    dict_out["path"].append(img_path)

    dict_out["class"].append(cls)
    dict_out["exam_class"].append(cls)

df_out = pd.DataFrame.from_dict(dict_out)
df_out.head()

,id,exam_id,path,class,exam_class
0,50a418190bc3fb1ef1633bf9678929b3,50a418190bc3fb1ef1633bf9678929b3,./train-png/50a418190bc3fb1ef1633bf9678929b3.png,no findings,no findings
1,21a10246a5ec7af151081d0cd6d65dc9,21a10246a5ec7af151081d0cd6d65dc9,./train-png/21a10246a5ec7af151081d0cd6d65dc9.png,no findings,no findings
2,9a5094b2563a1ef3ff50dc5c7ff71345,9a5094b2563a1ef3ff50dc5c7ff71345,./train-png/9a5094b2563a1ef3ff50dc5c7ff71345.png,"cardiomegaly,pleural effusion,pleural thickeni...","cardiomegaly,pleural effusion,pleural thickeni..."
3,051132a778e61a86eb147c7c6f564dfe,051132a778e61a86eb147c7c6f564dfe,./train-png/051132a778e61a86eb147c7c6f564dfe.png,"aortic enlargement,cardiomegaly,pleural thicke...","aortic enlargement,cardiomegaly,pleural thicke..."
4,063319de25ce7edb9b1c6b8881290140,063319de25ce7edb9b1c6b8881290140,./train-png/063319de25ce7edb9b1c6b8881290140.png,no findings,no findings


In [9]:
sub_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
45863,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R15,NaN,NaN,NaN,NaN
57424,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R16,NaN,NaN,NaN,NaN


In [13]:
print(df_out["class"].value_counts())

class
no findings                                                                                 10606
aortic enlargement,cardiomegaly                                                               357
cardiomegaly,aortic enlargement                                                               352
aortic enlargement                                                                            142
cardiomegaly                                                                                   94
                                                                                            ...  
pleural thickening,nodule/mass,aortic enlargement                                               1
lung opacity,cardiomegaly,aortic enlargement,pulmonary fibrosis,pleural thickening              1
lung opacity,pleural thickening,cardiomegaly,aortic enlargement,nodule/mass,other lesion        1
nodule/mass,infiltration,consolidation,ild,pulmonary fibrosis,pleural thickening                1
cardiomegaly,i

In [14]:
import numpy as np

df_out["dataset"] = "vinbigdata-cxr"
df_out["class_num"] = 14
df_out["contains_negative"] = 1
df_out["modality"] = "xray"

df_out["split"] = "test"

In [15]:
df_out["split"].value_counts()

split
test    15000
Name: count, dtype: int64

In [17]:
df_out.to_csv(f"{root}/metadata.csv", index=False)

In [38]:
print(df_out["class"].value_counts())

class
no findings                                                                                 10606
aortic enlargement,cardiomegaly                                                               357
cardiomegaly,aortic enlargement                                                               352
aortic enlargement                                                                            142
cardiomegaly                                                                                   94
                                                                                            ...  
pleural thickening,nodule/mass,aortic enlargement                                               1
lung opacity,cardiomegaly,aortic enlargement,pulmonary fibrosis,pleural thickening              1
lung opacity,pleural thickening,cardiomegaly,aortic enlargement,nodule/mass,other lesion        1
nodule/mass,infiltration,consolidation,ild,pulmonary fibrosis,pleural thickening                1
cardiomegaly,i

In [35]:
classes = dict()

for cls in df_out["class"].tolist():
    for x in cls.split(","):
        if x not in classes.keys():
            classes[x] = 1
        else:
            classes[x] += 1

print(classes)

{'no findings': 10606, 'cardiomegaly': 2300, 'pleural effusion': 1032, 'pleural thickening': 1981, 'aortic enlargement': 3067, 'pulmonary fibrosis': 1617, 'ild': 386, 'nodule/mass': 826, 'other lesion': 1134, 'lung opacity': 1322, 'infiltration': 613, 'consolidation': 353, 'calcification': 452, 'atelectasis': 186, 'pneumothorax': 96}


In [37]:
list(classes.keys())

['no findings',
 'cardiomegaly',
 'pleural effusion',
 'pleural thickening',
 'aortic enlargement',
 'pulmonary fibrosis',
 'ild',
 'nodule/mass',
 'other lesion',
 'lung opacity',
 'infiltration',
 'consolidation',
 'calcification',
 'atelectasis',
 'pneumothorax']